In [ ]:
### Install and import libraries
!pip install pandas openpyxl
!pip install dataset
!pip install unsloth
!pip install gradio
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

import pandas as pd
import json
from google.colab import files
import numpy as np
from datasets import Dataset
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import Dataset,concatenate_datasets

Creation of API (Unified Approach) 

In [ ]:
"""
This module processes multiple types of questions (True/False, Open-Ended, and Multiple-Choice) and provides utilities for structured data extraction and interaction via a Gradio interface.

Features:
- Structured data parsing from XML-like, Markdown-like, and raw text formats.
- True/False evaluation using a fine-tuned model.
- Open-Ended question answering with detailed responses.
- Multiple-Choice question answering with selected choice and justification.

Modules:
    - unsloth: Provides FastLanguageModel for efficient model inference.
    - torch: For PyTorch-based tensor operations.
    - gradio: For creating the user-friendly interface.
    - xml.etree.ElementTree: For XML parsing.
    - re: For regular expressions to process structured text.

Functions:
    extract_from_xml_et(text): Parses XML-like strings.
    extract_choice(text): Extracts the choice label (e.g., A), B)).
    extract_justification(text): Extracts the justification text.
    extract_from_markdown_regex(text): Extracts choice and justification from Markdown-like text.
    extract_fields(text): Combines various parsing methods for structured data extraction.
    generate_response_true_false(instruction): Evaluates True/False questions.
    generate_response_open_ended(instruction): Answers Open-Ended questions.
    generate_response_multiple_choice(question, choice_A, choice_B, choice_C, choice_D): Handles Multiple-Choice questions.
    true_false_greet(question): Interface function for True/False questions.
    open_ended_greet(question): Interface function for Open-Ended questions.
    multiple_choice_greet(question, choice_A, choice_B, choice_C, choice_D): Interface function for Multiple-Choice questions.
"""

from unsloth import FastLanguageModel
import torch
import gradio as gr
import xml.etree.ElementTree as ET
import re

def extract_from_xml_et(text: str) -> dict:
    """
    Parses an XML-like string and extracts key-value pairs from its elements.

    Args:
        text (str): A string containing XML-like content (e.g., <tag>value</tag>).

    Returns:
        dict: A dictionary where keys are lowercase XML tags and values are their text content.
        None: Returns None if XML parsing fails.

    Example:
        >>> text = '<key>"value"</key>'
        >>> extract_from_xml_et(text)
        {'key': 'value'}
    """
    try:
        wrapped_text = f"<root>{text}</root>"
        root = ET.fromstring(wrapped_text)
        data = {}
        for child in root:
            if child.text:
                value = child.text.strip().strip('"')
                data[child.tag.lower()] = value
        return data
    except ET.ParseError:
        return None

def extract_choice(text: str) -> str:
    """
    Extracts the choice (e.g., A), B)) from a text block.

    Args:
        text (str): Input text to search for the choice.

    Returns:
        str: The extracted choice, or None if not found.

    Example:
        >>> text = "A) This is a sample choice."
        >>> extract_choice(text)
        'A)'
    """
    choice_pattern = r'([A-D]\))'
    match = re.search(choice_pattern, text)
    if match:
        return match.group(1).strip()
    return None

def extract_justification(text: str) -> str:
    """
    Extracts the justification text from a text block.

    Args:
        text (str): Input text to search for the justification.

    Returns:
        str: The extracted justification, or None if not found.

    Example:
        >>> text = "- Justification: This is the reason."
        >>> extract_justification(text)
        'This is the reason.'
    """
    justification_pattern = r'(?:- )?Justification:\s*(.+)'
    match = re.search(justification_pattern, text)
    if match:
        return match.group(1).strip()
    return None

def extract_from_markdown_regex(text: str) -> dict:
    """
    Extracts structured data from Markdown-like text blocks.

    Args:
        text (str): Input text containing Markdown-like content with **choice** and **justification** fields.

    Returns:
        dict: A dictionary containing "choice" and "justification", or None if no match is found.

    Example:
        >>> text = "**choice**: A **justification**: This is the reason."
        >>> extract_from_markdown_regex(text)
        {'choice': 'A', 'justification': 'This is the reason.'}
    """
    choice_pattern = r'\*\*choice\*\*:\s*(.+?)'
    justification_pattern = r'\*\*justification\*\*:\s*([\s\S]+?)(?=\*\*choice\*\*|$)'
    choice_match = re.search(choice_pattern, text)
    justification_match = re.search(justification_pattern, text)

    if choice_match and justification_match:
        return {
            "choice": choice_match.group(1).strip(),
            "justification": justification_match.group(1).strip()
        }
    return None

def extract_fields(text: str) -> list:
    """
    Processes text blocks to extract structured data using various methods.

    Args:
        text (str): Input text containing one or more blocks of data.

    Returns:
        list: A list of dictionaries, each containing extracted data from a block.

    Workflow:
        - Splits the input text into blocks using double line breaks (\n\n).
        - Tries XML parsing, regex for choice/justification, and Markdown-like parsing.
    """
    entries = []
    blocks = re.split(r'\n\s*\n', text.strip())

    for block in blocks:
        extracted_data = {}

        # Try extracting using XML
        xml_data = extract_from_xml_et(block)
        if xml_data:
            entries.append(xml_data)
            continue

        # Try extracting using choice and justification regex
        choice = extract_choice(block)
        justification = extract_justification(block)
        if choice or justification:
            extracted_data["choice"] = choice
            extracted_data["justification"] = justification
            entries.append(extracted_data)
            continue

        # Try extracting using Markdown regex
        markdown_data = extract_from_markdown_regex(block)
        if markdown_data:
            entries.append(markdown_data)

    return entries

### Model Initialization ###
model, tokenizer = FastLanguageModel.from_pretrained('./unified_model')

def generate_response_true_false(instruction: str) -> str:
    """
    Generates a "True" or "False" response based on the provided statement.

    Args:
        instruction (str): A string containing the statement to evaluate.

    Returns:
        str: "True" or "False", or "Unable to determine" if parsing fails.
    """
    FastLanguageModel.for_inference(model)
    prompt = f"""### Instruction:
    Determine if the following statement is true or false. Respond only with "True" or "False".

    ### Statement:
    {instruction}

    ### Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.split("### Answer:")[-1].strip()

    if response.lower() == "true":
        return "True"
    elif response.lower() == "false":
        return "False"
    return "Unable to determine."

# Function to generate responses for open-ended questions
def generate_response_open_ended(instruction):
    """
    Generates a response using your fine-tuned model based on the provided instruction.

    This function enables faster inference through the `FastLanguageModel` and prepares a
    prompt for the model to answer the provided question.

    Args:
        instruction (str): A string containing the statement and instructions to be evaluated.

    Returns:
        str: A response from the model to the provided question or "Unable to determine" if the
             response cannot be parsed reliably.
    """
    FastLanguageModel.for_inference(model)  # Enable faster inference within the function

    # Create the prompt for the model
    prompt = f"""### Instruction:
    Answer the provided question with the knowledge provided to you
    ### Question:
    {instruction}

    ### Answer:
    """

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            early_stopping=False,
            min_length=50,
            length_penalty=2,
            max_length=200
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the answer from the generated response
    response = response.split('### Answer:')[1]
    return response

# Function to generate responses for multiple-choice questions
def generate_response_multiple_choice(question, choice_A, choice_B, choice_C, choice_D):
    """
    Generates a response using a fine-tuned language model for multiple-choice questions.

    Args:
        question (str): The question to be answered.
        choice_A (str): Option A.
        choice_B (str): Option B.
        choice_C (str): Option C.
        choice_D (str): Option D.

    Returns:
        dict: A dictionary with the selected choice and its justification.
              Example:
              {
                  "choice": "A",
                  "justification": "Explanation for why Option A is correct."
              }
              Defaults to:
              {
                  "choice": "None",
                  "justification": "Could not parse JSON."
              } if parsing fails.
    """
    instruction = f'''{question}
  Choices:
  A) {choice_A},
  B) {choice_B},
  C) {choice_C},
  D) {choice_D}
    '''

    FastLanguageModel.for_inference(model)  # Enable faster inference

    # Define the prompt
    prompt = f"""### Instruction:
    In the following question, you are provided with 4 choices. Select the best choice based on the knowledge provided and provide a justification for that choice.

    **You must return only your response with the following keys:**
      - "choice": The best choice letter
      - "justification": The justification for your choice

    **Example Response:**
      **choice**: A
      **justification**: Explanation for why Option A is correct

    ### Question:
    {instruction}

    ### Answer:
    """

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate a response from the model
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            early_stopping=True,
            min_length=50,
            length_penalty=2,
            do_sample=True,
            max_new_tokens=300,
            top_p=0.95,
            top_k=50,
            temperature=0.65,
            num_return_sequences=1
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the answer from the generated response
    response = response.split('### Answer:')[1]
    print("RESPONSE", response)
    data = extract_fields(response)
    if len(data) == 0:
        response = {"choice": "None", "justification": "Could not parse JSON"}
    else:
        response = {"choice": data[-1]['choice'], "justification": data[-1]['justification']}
    return response

# Helper functions for different question types
def true_false_greet(question):
    """
    Responds to a True/False question or returns a default response if no question is provided.

    Args:
        question (str): The input True/False question.

    Returns:
        str: The response from the model or a default message.
    """
    if question == "":
        return "No question was given to answer"
    else:
        response = generate_response_true_false(question)  # Placeholder
        return f"{response}!"

def open_ended_greet(question):
    """
    Processes an open-ended question and returns a response.

    Args:
        question (str): The input question provided by the user.

    Returns:
        str: The response from the model or a default message.
    """
    if question == "":
        return "No question was given to answer"
    else:
        response = generate_response_open_ended(question)
        return f"{response}!"

def multiple_choice_greet(question, choice_A, choice_B, choice_C, choice_D):
    """
    Processes a multiple-choice question and returns a response.

    Args:
        question (str): The input question.
        choice_A (str): Option A.
        choice_B (str): Option B.
        choice_C (str): Option C.
        choice_D (str): Option D.

    Returns:
        str: The response, including the selected choice and its justification.
    """
    if question == "":
        return "No question was given to answer"
    if choice_A == "" and choice_B == "" and choice_C == "" and choice_D == "":
        return "No choice was given"
    else:
        response = generate_response_multiple_choice(question, choice_A, choice_B, choice_C, choice_D)
        actual_response = f"Selected Choice: {response['choice']}\nJustification: {response['justification']}"
        return actual_response

# Functions to toggle visibility of interfaces
def show_true_false_interface():
    return gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)

def show_open_ended_interface():
    return gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)

def show_multiple_choice_interface():
    return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)

# Gradio UI setup
with gr.Blocks() as demo:

    # Navigation buttons
    with gr.Row():
        btn_t_f = gr.Button('True/False questions')
        btn_open_ended = gr.Button('Open-Ended questions')
        btn_m_c = gr.Button('Multiple-Choice questions')

    # True/False questions interface
    with gr.Column(visible=True) as true_false_interface:
        gr.Markdown("## True-False Template")
        question_simple = gr.Textbox(label="Enter your question")
        simple_output = gr.Textbox(label="Output", interactive=False)
        submit_simple = gr.Button("Submit")
        submit_simple.click(true_false_greet, inputs=question_simple, outputs=simple_output)

    # Open-ended questions interface
    with gr.Column(visible=False) as open_ended_interface:
        gr.Markdown("## Open Ended Template")
        question_simple = gr.Textbox(label="Enter your question")
        simple_output = gr.Textbox(label="Output", interactive=False)
        submit_simple = gr.Button("Submit")
        submit_simple.click(open_ended_greet, inputs=question_simple, outputs=simple_output)

    # Multiple-choice questions interface
    with gr.Column(visible=False) as mc_interface:
        gr.Markdown("## Multiple-Choice Template")
        question_mc = gr.Textbox(label="Enter your question")
        choice_A = gr.Textbox(label="Choice A")
        choice_B = gr.Textbox(label="Choice B")
        choice_C = gr.Textbox(label="Choice C")
        choice_D = gr.Textbox(label="Choice D")
        mc_output = gr.Textbox(label="Output", interactive=False)
        submit_mc = gr.Button("Submit")
        submit_mc.click(multiple_choice_greet, inputs=[question_mc, choice_A, choice_B, choice_C, choice_D], outputs=mc_output)

    # Navigation button functionality
    btn_t_f.click(show_true_false_interface, outputs=[true_false_interface, open_ended_interface, mc_interface])
    btn_open_ended.click(show_open_ended_interface, outputs=[true_false_interface, open_ended_interface, mc_interface])
    btn_m_c.click(show_multiple_choice_interface, outputs=[true_false_interface, open_ended_interface, mc_interface])

# Launch the Gradio app
demo.launch()



Creation of API (Chat-Bot Approach)

In [ ]:
# Import necessary libraries
from unsloth import FastLanguageModel
import torch
import gradio as gr

# Provide a reference to documentation for further information
"""
For more information on `huggingface_hub` Inference API support, please check the docs: https://huggingface.co/docs/huggingface_hub/v0.22.2/en/guides/inference
"""

# Load the pre-trained language model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained('./unified_model')

# Function to generate a response
def generate_response(instruction, chat_history):
    """
    Generates a response using your fine-tuned model.

    Args:
        instruction (str): The user's input question or instruction.
        chat_history (dict): A dictionary maintaining the conversation history.

    Returns:
        str: The generated response from the model.
    """
    # Create a prompt for the model
    prompt = f"""### Instruction:
    Answer the following question.

    ### Question:
    {instruction}

    Provide a unique, concise, and non-repetitive answer.
    ### Answer:"""

    # Tokenize the prompt and move it to GPU for inference
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            early_stopping=True,
            min_length=50,
            length_penalty=2,
            do_sample=True,
            max_new_tokens=300,
            top_p=0.95,
            top_k=50,
            temperature=0.7,
            repetition_penalty=1.2,
            num_return_sequences=1
        )

    # Decode the generated response and extract the answer
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.split("### Answer:")[-1]
    return response

# Function to update chat history
def update_chat_history(chat_history, user_message, bot_message):
    """
    Updates the chat history to maintain relevance and avoid excessive growth.

    Args:
        chat_history (dict): A dictionary containing user and bot messages.
        user_message (str): The latest message from the user.
        bot_message (str): The latest response from the bot.

    Returns:
        dict: The updated chat history, trimmed to the last N interactions.
    """
    chat_history['user'].append(user_message)
    chat_history['bot'].append(bot_message)

    # Keep only the last 5 interactions for brevity
    if len(chat_history['user']) > 5:
        chat_history['user'] = chat_history['user'][-5:]
        chat_history['bot'] = chat_history['bot'][-5:]

    return chat_history

# Main chatbot function
def chatbot(input_text, chat_history):
    """
    Handles user input, generates a response, and updates the chat history.

    Args:
        input_text (str): The user's input message.
        chat_history (list): A list of tuples containing previous user and bot messages.

    Returns:
        str: An empty string (clears the input box).
        list: The updated chat history as a list of tuples.
    """
    # Initialize a structured dictionary for conversation messages
    messages = {
        "user": [],
        "bot": [],
    }

    # Populate messages with existing chat history
    for user_msg, bot_msg in chat_history:
        messages["user"].append(user_msg)
        messages["bot"].append(bot_msg)

    # Generate a response based on user input
    bot_response = generate_response(input_text, messages)

    # Update the chat history with the new messages
    chat_history.append(("User: " + input_text, bot_response))
    messages = update_chat_history(messages, input_text, bot_response)

    return "", chat_history

# Build the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown('## AILA INTERFACE DEMO')

    # Input section for user messages
    with gr.Row():
        user_input = gr.Textbox(
            placeholder="Type your message here...",
            label="Your Message",
            lines=1
        )

    # Button to submit user input
    submit_button = gr.Button('Submit')

    # Display chat history
    chat_history = gr.Chatbot()

    # Link the button to the chatbot function
    submit_button.click(
        chatbot,
        inputs=[user_input, chat_history],
        outputs=[user_input, chat_history]
    )

# Launch the interface
demo.launch()
